In [ ]:
# 설명 : 동적페이지에 키워드가 있는 지 확인 -> 앞에서 찾은 부분이 정적 페이지에 있는 지 확인 - > 있으면 추출

In [17]:
# keyword 예시 : ~남음, ~개, ~보고 있~, ~까지, ~명이 담음, ~보는 

In [43]:
#pandas import
import pandas as pd
import time
import os

#BeautifulSoup import
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

#Selenium import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException  # 예외 처리 추가
from selenium.webdriver.chrome.options import Options # user-agent 변경

In [44]:
# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_argument("Mozilla/5.0")  # 원하는 User-Agent 문자열로 변경

In [45]:
# 가장 작은 div 레이아웃 가져오기
def find_div(elements):
    html = []
    for element in elements:
        try:
            # 현재 요소를 포함하는 가장 가까운 div 찾기
            closest_div = element.find_element(By.XPATH, './ancestor::div[1]')
            html_code = closest_div.get_attribute('outerHTML')
            
            # 결과 출력
            print("HTML Code:")
            print(html_code)
            
            #결과값 저장
            html.append(html_code)
            
        except NoSuchElementException:
            print("가장 가까운 div 요소를 찾을 수 없습니다.")
            
    return html

In [46]:
# 동적 페이지와 동일한 부분 가져오기
def false_detect(dynamic_html,static_html):
    dynamic = []
    static = []
    result = []
    for dynamic_block in dynamic_html:

        false_result = "false"
        static_parent = None
        #동적 요소에서 텍스트만 추출
        dynamic_soup = BeautifulSoup(dynamic_block, "html.parser")
        dynamic_text = dynamic_soup.get_text(strip=True)
        
        #동적 요소의 텍스트를 기반으로 동일한 내용 찾기
        static_element = static_html.find(string=lambda text: dynamic_text in text if text else False)
        
        if static_element:
            static_parent = static_element.find_parent() #텍스트가 속한 부모 태그를 추출
            print(f"---동일한 정보가 존재합니다.--- | dynamic : {dynamic_block} | static : {static_parent}")
            
        else:
            print(f"---해당 정보가 없습니다.--- | dynamic : {dynamic_block}")
            false_result = "true"
            
        dynamic.append(dynamic_block)
        static.append(static_parent)
        result.append(false_result)
        print("------------------------")
        
    return dynamic, static, result

In [47]:
data = pd.read_csv('false_test.csv')
driver = webdriver.Chrome()

# 결과 파일 존재 여부 확인
output_file = "false_result.csv"
file_exists = os.path.isfile(output_file)

for url in data['URL']:
    #1. url을 연다
    print("Dynamic URL open : "+ url)
    driver.get(url) #동적 페이지
    
    time.sleep(3)
    
    #2. element 불러오기(동적)
    keyword = "남음"
    dynamic_elements = driver.find_elements(By.XPATH, f"//*[contains(text(), '{keyword}')]")
    
    #3. div 레이아웃 가져오기(동적)
    dynamic_html = find_div(dynamic_elements)
    
    #4. 정적 페이지에서 해당 레이아웃 찾기
    print("Static URL open: "+ url)
    static_html = urlopen(url) 
    bs_obj = BeautifulSoup(static_html, "html.parser") #정적페이지
    result_dynamic, result_static, result = false_detect(dynamic_html, bs_obj) #동일한 부분 찾기
    
    #5. 데이터 저장
    df = pd.DataFrame({'Dynamic_HTML': result_dynamic, 'Static_HTML':result_static, 'Result': result})
    if not file_exists:
        df.to_csv(output_file, index=False, mode='w', header=True, encoding='utf-8')
        file_exists = True
    else:
        df.to_csv(output_file, index=False, mode='a', header=False, encoding='utf-8')
    

Dynamic URL open : https://www.agoda.com/ko-kr/search?city=16527&locale=ko-kr&ckuid=cbe5b914-1610-41bd-ab4d-111db75912cd&prid=0&currency=KRW&correlationId=5e3ddd71-186f-4493-85f2-2bc2a95ae2ad&analyticsSessionId=4589510054972858855&pageTypeId=1&realLanguageId=9&languageId=9&origin=KR&stateCode=11&cid=1844104&userId=cbe5b914-1610-41bd-ab4d-111db75912cd&whitelabelid=1&loginLvl=0&storefrontId=3&currencyId=26&currencyCode=KRW&htmlLanguage=ko-kr&cultureInfoName=ko-kr&machineName=hk-pc-2i-acm-web-user-6599bbcc8d-h9k4g&trafficGroupId=1&trafficSubGroupId=84&aid=130589&useFullPageLogin=true&cttp=4&isRealUser=true&mode=production&browserFamily=Chrome&cdnDomain=agoda.net&checkIn=2024-10-21&checkOut=2024-10-25&rooms=1&adults=2&children=0&priceCur=KRW&los=4&textToSearch=%ED%9B%84%EC%BF%A0%EC%98%A4%EC%B9%B4&travellerType=1&familyMode=off&ds=l%2Bf9mEYMkCznDxnr&productType=-1
HTML Code:
<div data-element-name="discount-percent" data-discount-percentage="0" data-badge-id="disc-perc" class="a46d6-box a46

HTTPError: HTTP Error 403: Forbidden

In [48]:
driver.quit()